In [1]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

In [11]:
import os
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
QgO4bLzRTe4eveK4SozameJiPT4vAEbEP0XftIglpGAapGHG


In [13]:
# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url=f"https://cg.optimizely.com/content/v2?auth={os.getenv('SINGLE_KEY')}")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)


Go to properties and then go to all the fields you want to be searchable.
Then change the indexing property of each of those to searchable.
Retrigger the graphql sync


If you don't do this then the content won't show up in _fulltext

In [10]:
query = gql(
    """
    query AcrossAllContent {
    _Page {
        items {
        _fulltext
        _metadata {
            url {
            base
            hierarchical
            }
            displayName
        }
        _id
        }
    }
    }
"""
)

# Execute the query on the transport
result = await client.execute_async(query)
print(result)

{'_Page': {'items': [{'_fulltext': ['Image', 'All your money in one place', 'Heading', "Your cash, your choices, and a banking experience that's as unique as you are", 'Call To Action', 'banking made modern', 'Send, spend and save smarter', 'Change the way you bank.', 'Saving', 'Grow your money every day', '<p>Watch your savings grow, not your worries. Always high interest rates because you deserve it. Oh, and no fees ever. Did we mention you deserve it?&nbsp;</p>', 'Paragraph', 'Checking', 'Banking without the bull', '<p>Checking that works for you to satisfy your checking account needs.&nbsp;</p>', 'Credit cards', 'More points, no problem', '<p>Find the perfect credit card from among our most popular options.</p>', 'Loans', 'Funds in a flash', '<p>Need something now? Fast approvals and low rates &ndash; your shortcut to making your dreams happen.</p>', 'Trusted by everyone you trust', 'Join 3 million+ happy people who already bank with Mosey', 'Moseybank meets you where you are, no m

In [31]:
formatted_items = []
for item in result["_Page"]["items"]:
    print(item["_metadata"]["displayName"])
    print(f'{item["_metadata"]["url"]["base"]}{item["_metadata"]["url"]["hierarchical"]}')
    print(' '.join(item["_fulltext"]))
    print(item["_id"])
    obj = {
        "id": item["_id"],
        "title_t": item["_metadata"]["displayName"],
        "uri": f'{item["_metadata"]["url"]["base"]}{item["_metadata"]["url"]["hierarchical"]}',
        "content_t": ' '.join(item["_fulltext"])
    }
    formatted_items.append(obj)
    
    print("\n\n")

Mosey Bank
https://optimizely-saas-5q4iokzv8-karans-projects-434d3075.vercel.app/en/
Image All your money in one place Heading Your cash, your choices, and a banking experience that's as unique as you are Call To Action banking made modern Send, spend and save smarter Change the way you bank. Saving Grow your money every day <p>Watch your savings grow, not your worries. Always high interest rates because you deserve it. Oh, and no fees ever. Did we mention you deserve it?&nbsp;</p> Paragraph Checking Banking without the bull <p>Checking that works for you to satisfy your checking account needs.&nbsp;</p> Credit cards More points, no problem <p>Find the perfect credit card from among our most popular options.</p> Loans Funds in a flash <p>Need something now? Fast approvals and low rates &ndash; your shortcut to making your dreams happen.</p> Trusted by everyone you trust Join 3 million+ happy people who already bank with Mosey Moseybank meets you where you are, no matter who you are. Te

In [17]:
from lxml.html.clean import Cleaner
from lxml import etree

In [14]:
is_clean_javascript = True
is_clean_style = True
kill_tags = ["noscript", "footer", "header", "nav", "button", "form"]

In [21]:
def dom_cleaner(content):
    cleaner = Cleaner(page_structure=False, links=False, style=is_clean_style,javascript=is_clean_javascript, kill_tags=kill_tags)
    
    return cleaner.clean_html(content)

In [33]:
for item in formatted_items:
    tree = etree.HTML(dom_cleaner(item["content_t"]))
    item["content"] = " ".join(tree.xpath("//text()"))
    print(item["content"])

Image All your money in one place Heading Your cash, your choices, and a banking experience that's as unique as you are Call To Action banking made modern Send, spend and save smarter Change the way you bank. Saving Grow your money every day  Watch your savings grow, not your worries. Always high interest rates because you deserve it. Oh, and no fees ever. Did we mention you deserve it?   Paragraph Checking Banking without the bull  Checking that works for you to satisfy your checking account needs.   Credit cards More points, no problem  Find the perfect credit card from among our most popular options.  Loans Funds in a flash  Need something now? Fast approvals and low rates – your shortcut to making your dreams happen.  Trusted by everyone you trust Join 3 million+ happy people who already bank with Mosey Moseybank meets you where you are, no matter who you are. Testimonial Mosey anywhere One app to bank it all  Delete all your other banking apps (jk... unless?) Mosey's app does it a

In [28]:
import requests
import urllib.parse as urlparse

In [38]:
def get_path(url_string):
    url_parse = urlparse.urlparse(url_string)
    path_str = url_parse.path.strip("/").replace("/", " / ")
    if not path_str:
        path_str = url_parse.netloc
    return path_str


def get_type_from_url(url_string):
    url_parse = urlparse.urlparse(url_string)
    try:
        pagetype = url_parse.path.strip("/").split("/")[1].title()
        if "-" in pagetype:
            pagetype = " ".join(pagetype.split("-")).title()
        if "_" in pagetype:
            pagetype = " ".join(pagetype.split("_")).title()
        if not pagetype:
            return "Web Page"
        return pagetype
    except IndexError:
        return "Web Page"

In [39]:
for item in formatted_items:
    item["typeurl_s"] = get_type_from_url(item["uri"])
    item["path_s"] = get_path(item["uri"])

In [40]:
headers = {
    "Authorization":f"Token {os.getenv('SITE_SEARCH_RW_TOKEN')}"
}
resp = requests.post("https://searchcloud-2-us-east-1.searchstax.com/29847/optimizelypoc-1946/update", headers=headers, json=formatted_items)
print(resp.json())

{'responseHeader': {'rf': 2, 'status': 0, 'QTime': 28}}
